# Data processing

This example walks through the basics for processing data and added metrics.

## Concepts

Devices in the framework contain _raw readings_ that are under the device.readings pandas dataframe. A list of the sensors raw metrics can be shown in device.sensors.

Devices can also contain processed values called metrics. These metrics can be added by passing a callable function and then processed.

In [ ]:
from scdata.test import Test
from scdata.device import Device
from scdata._config import config

In [ ]:
config._log_level='INFO'

In [ ]:
test = Test(name='EXAMPLE')

In [ ]:
await test.load()

## Process basics

In [ ]:
for device in test.devices: 
    print (device.id)

In [ ]:
## The readings for each device are accessible via
test.get_device(16784).data

## Basic example calculation

In [ ]:
df = test.get_device(16871).data

In [ ]:
df['METRIC'] = 8 * df['TEMP'] + 25 * df['PRESS']

In [ ]:
df[['TEMP', 'PRESS', 'METRIC']]

## Making it repeatable

In [ ]:
## The metrics for each device are accessible via
test.get_device(16784).metrics

In [ ]:
help(Test.process)

In [ ]:
d = test.get_device(16871)

In [ ]:
## Process the metrics as a default
test.process()

Now we can see processed metrics in the `device.readings`

In [ ]:
for device in test.devices:
    print (device.data.columns)

## Add metrics

In [ ]:
help(Device.add_metric)

In [ ]:
help(Device.process)

In [ ]:
import scdata
help(scdata.device.process.timeseries)
# help(sc.device.process.alphasense)
# help(sc.device.process.regression)

### Basic polynomial

In [ ]:
help(scdata.device.process.timeseries.poly_ts)

In [ ]:
from scdata.models import Metric

In [ ]:
metric = Metric(name='TP_Poly',
                description='Basic Polynomial calculation',
                function='poly_ts',
                kwargs= {'channels': ['TEMP', 'PRESS'], 'coefficients': [8, 25]}
               )

test.get_device(16871).add_metric(metric)

In [ ]:
test.get_device(16871).process(lmetrics=['TP_Poly'])

In [ ]:
test.get_device(16871).data.loc[:,['TEMP', 'PRESS', 'TP_Poly', 'METRIC']]

In [ ]:
traces = {1: {'devices': 16871,
              'channel': 'TP_Poly',
              'subplot': 2},
          2: {'devices': 16871,
              'channel': 'TEMP',
              'subplot': 1},
          3: {'devices': 16871,
              'channel': 'PRESS',
              'subplot': 1},            
         }

options = {
            'frequency': '1H'
}
formatting = {'width': 800, 'height': 200, 'padding-bottom': 400}
test.ts_uplot(traces = traces, options = options, formatting=formatting)

### Basic smoothing

In [ ]:
help(scdata.device.process.timeseries.rolling_avg)

In [ ]:
metric = Metric(name='NOISE_A_SMOOTH',
                description='Basic smoothing calculation',
                function='rolling_avg',
                kwargs= {'name': ['NOISE_A'], 'window_size': 5}
               )
test.get_device(16871).add_metric(metric)

In [ ]:
test.get_device(16871).process(lmetrics=['NOISE_A_SMOOTH'])

In [ ]:
metric = Metric(name='NOISE_A_SMOOTH_10',
                description='Basic smoothing calculation',
                function='rolling_avg',
                kwargs= {'name': ['NOISE_A'], 'window_size': 10}
               )
test.get_device(16871).add_metric(metric)

In [ ]:
test.get_device(16871).process(lmetrics=['NOISE_A_SMOOTH_10'])

In [ ]:
metric = Metric(name='NOISE_A_SMOOTH_60',
                description='Basic smoothing calculation',
                function='rolling_avg',
                kwargs= {'name': ['NOISE_A'], 'window_size': 60}
               )
test.get_device(16871).add_metric(metric)

In [ ]:
test.get_device(16871).process(lmetrics=['NOISE_A_SMOOTH_60'])

In [ ]:
test.get_device(16871).data.columns

In [ ]:
traces = {1: {'devices': 16871,
              'channel': 'NOISE_A',
              'subplot': 1},
          2: {'devices': 16871,
              'channel': 'NOISE_A_OUTLIERS',
              'subplot': 1},
          3: {'devices': 16871,
              'channel': 'NOISE_A_SMOOTH_10',
              'subplot': 1},
          4: {'devices': 16871,
              'channel': 'NOISE_A_SMOOTH_60',
              'subplot': 1},
          5: {'devices': 16871,
              'channel': 'TEMP',
              'subplot': 2} 
         }

options = {
            'frequency': '1Min'
}
formatting = {'width': 800, 'height': 400}
test.ts_uplot(traces = traces, options = options, formatting=formatting)

## Reprocessing

When adding a new metric, one can only process the added metric as above or the whole test: `test.process()`

If processes take too long, when adding a metric, the new ones can be processed as: `test.process(only_new = True)`

In [ ]:
help(Test.process)

In [ ]:
help(scdata.device.process.timeseries.clean_ts)

In this example, we will remove values between 35-50dBA and perform a rolling average on the data that is left:

In [ ]:
metric = Metric(name='NOISE_A_CL',
                description='Clean Data calculation',
                function='clean_ts',
                kwargs= {'name': 'NOISE_A', 'limits': [35, 50], 'window_size': 3}
               )
test.get_device(16871).add_metric(metric)

In [ ]:
test.process(only_new = True)

In [ ]:
test.get_device(16871).data.loc[:,['NOISE_A', 'NOISE_A_CL']]

In [ ]:
traces = {1: {'devices': 16871,
              'channel': 'NOISE_A',
              'subplot': 1},
          2: {'devices': 16871,
              'channel': 'NOISE_A_CL',
              'subplot': 1},          
         }

options = {
            'frequency': '1Min'
}
test.ts_uplot(traces = traces, options = options)